# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [1]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t

In [2]:
# rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
rc = ut.read_config("../config/test.toml")
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [3]:
import vipy.simulation.scan as scan

In [4]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [5]:
time = ut.calc_time_steps(rc)
print(time)

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:20:00.000'
 '2016:095:00:20:10.000' '2016:095:00:40:00.000' '2016:095:00:40:10.000'
 '2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:20:00.000'
 '2016:095:01:20:10.000' '2016:095:01:20:20.000']


## Calculate Baselines for one measurement

In [6]:
baselines = scan.get_baselines(src_crd, time[(0):(31)], array_layout)

## Calculate Mueller/Jones matrices

In [7]:
import numpy as np
import torch

In [8]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

/home/kschmidt_local/.local/anaconda3/envs/vipy/lib/python3.9/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/kschmidt_local/radio_nets/vipy/vipy/simulation/scan.py:319: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  K = torch.tensor(getK(baselines, lm, wave))


CPU times: user 9.68 s, sys: 2.92 s, total: 12.6 s
Wall time: 2.19 s


## Open Source Image and compute Stokes vector

In [9]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [10]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [11]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = lm[255,0,0]-lm[0,0,0]
delta_m = lm[0,255,0]-lm[0,0,0]
print(I.shape)
print(JJ_f1.shape)

integral = scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

torch.Size([256, 256, 4])
torch.Size([256, 256, 86, 4, 4])
2.980687141418457
CPU times: user 3.08 s, sys: 1.21 s, total: 4.29 s
Wall time: 611 ms


In [12]:
integral.shape

torch.Size([58, 4])

## All scan loop

In [13]:
from tqdm import tqdm

In [14]:
torch.set_num_threads(12)

In [15]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [21]:
integral2 = np.array([])
for i in tqdm(range(5)):
    t = time[i*2:(i+1)*2+1]
    print(t)
    baselines = scan.get_baselines(src_crd, t, array_layout)
    print(len(baselines.u[baselines.valid==True]))
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)
    print(JJ_f1.shape)

    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
    delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
    print(delta_l)

    integral2 = np.append(integral2,scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m))
    print(scan.integrate(JJ_f1, JJ_f2, I, 28, delta_t, delta_f, delta_l, delta_m))


  0%|          | 0/5 [00:00<?, ?it/s]

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:20:00.000']
18
torch.Size([256, 256, 18, 4, 4])
0.0001830982550547064


 20%|██        | 1/5 [00:00<00:03,  1.33it/s]

tensor([], size=(0, 4), dtype=torch.complex128)
['2016:095:00:20:00.000' '2016:095:00:20:10.000' '2016:095:00:40:00.000']
18
torch.Size([256, 256, 18, 4, 4])
0.0001830982550547064


 40%|████      | 2/5 [00:01<00:02,  1.32it/s]

...

tensor([], size=(0, 4), dtype=torch.complex128)
['2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:20:00.000']
30
torch.Size([256, 256, 30, 4, 4])
0.0001830982550547064


 80%|████████  | 4/5 [00:03<00:00,  1.03it/s]

tensor([[-1.2060e-04-4.1527e-04j, -3.0622e-05-2.3859e-04j,
         -1.9777e-04-3.6566e-04j, -2.2173e-04-3.6071e-04j],
        [ 9.3382e-05-7.6143e-06j,  1.3393e-04-5.6732e-06j,
         -4.9896e-05-3.0807e-05j, -1.5496e-04-1.3255e-06j]],
       dtype=torch.complex128)
['2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000']
30
torch.Size([256, 256, 30, 4, 4])
0.0001830982550547064


100%|██████████| 5/5 [00:04<00:00,  1.04it/s]

tensor([[ 3.8277e-04-4.8894e-05j,  4.2522e-04-3.9275e-05j,
         -1.4922e-04-1.6564e-05j, -2.8861e-04-8.6371e-06j],
        [-6.3456e-06-6.1825e-05j,  4.1874e-06-8.6059e-05j,
         -8.4928e-05+1.5697e-05j, -2.4825e-05+9.5655e-05j]],
       dtype=torch.complex128)


In [17]:
integral2.reshape(-1, 4)

array([[-1.20599473e-04-4.15274376e-04j, -3.06220003e-05-2.38589020e-04j,
        -1.97772117e-04-3.65660429e-04j, -2.21726247e-04-3.60708531e-04j],
       [ 9.33821084e-05-7.61432803e-06j,  1.33925201e-04-5.67315459e-06j,
        -4.98959871e-05-3.08073934e-05j, -1.54963344e-04-1.32553751e-06j],
       [ 3.82770825e-04-4.88944544e-05j,  4.25218126e-04-3.92750042e-05j,
        -1.49220453e-04-1.65644602e-05j, -2.88608101e-04-8.63706901e-06j],
       [-6.34555502e-06-6.18249744e-05j,  4.18736550e-06-8.60586008e-05j,
        -8.49278603e-05+1.56972482e-05j, -2.48247721e-05+9.56554435e-05j]])

In [18]:
I

tensor([[[4.2461e-07+0.j, 3.0025e-07+0.j, 3.0025e-07+0.j, 0.0000e+00+0.j],
         [2.1231e-07+0.j, 1.5012e-07+0.j, 1.5012e-07+0.j, 0.0000e+00+0.j],
         [6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         ...,
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j],
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j],
         [1.0615e-06+0.j, 7.5061e-07+0.j, 7.5061e-07+0.j, 0.0000e+00+0.j]],

        [[6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         [6.3692e-07+0.j, 4.5037e-07+0.j, 4.5037e-07+0.j, 0.0000e+00+0.j],
         [8.4922e-07+0.j, 6.0049e-07+0.j, 6.0049e-07+0.j, 0.0000e+00+0.j],
         ...,
         [8.4922e-07+0.j, 6.0049e-07+0.j, 6.0049e-07+0.j, 0.0000e+00+0.j],
         [3.1846e-06+0.j, 2.2518e-06+0.j, 2.2518e-06+0.j, 0.0000e+00+0.j],
         [8.4922e-06+0.j, 6.0049e-06+0.j, 6.0049e-06+0.j, 0.0000e+00+0.j]],

        [[4.2461e-07+0.j, 3.0025e-07+0.j, 3.0025e-07+0.j, 0.0000e+00